In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [0]:
filepath = 'dbfs:/FileStore/tables/newZomPySpark.csv'
#df = spark.read.option('mode','DROPMALFORMED').csv(filepath)
#df = pd.read_csv(filepath)


In [0]:
df = spark.read.format("csv").option("inferSchema","true").option("sep",",").option("header","true").load(filepath)

In [0]:
#df = df[df['url'].str.startswith("+") == False]
df.show()

+--------------------+------------+----------+-----+-----+------------+-------------------+--------------------+---------------------------+---------------+
|                name|online_order|book_table| rate|votes|    location|          rest_type|            cuisines|approx_cost(for two people)|listed_in(type)|
+--------------------+------------+----------+-----+-----+------------+-------------------+--------------------+---------------------------+---------------+
|               Jalsa|         Yes|       Yes|4.1/5|  775|Banashankari|      Casual Dining|South Indian, Mug...|                        800|         Buffet|
|      Spice Elephant|         Yes|        No|4.1/5|  787|Banashankari|      Casual Dining|Chinese, North In...|                        800|         Buffet|
|     San Churro Cafe|         Yes|        No|3.8/5|  918|Banashankari|Cafe, Casual Dining|Cafe, Mexican, It...|                        800|         Buffet|
|Addhuri Udupi Bho...|          No|        No|3.7/5|   88|

In [0]:
df = df.toPandas()
df.shape

(51718, 10)

In [0]:
df = df[df['approx_cost(for two people)'].str.contains("RATED") == False]
df.shape

(51372, 10)

In [0]:
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].astype(str).apply(lambda x: x.replace(',', ''))

In [0]:
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].astype(float)

In [0]:
df['approx_cost(for two people)'].unique()

array([ 800.,  300.,  600.,  700.,  550.,  500.,  450.,  650.,  400.,
        900.,  200.,  750.,  150.,  850.,  100., 1200.,  350.,  250.,
        950., 1000., 1500., 1300.,  199.,   80., 1100.,  160., 1600.,
        230.,  130.,   50.,  190., 1700., 1400.,  180., 1350., 2200.,
       2000., 1800., 1900.,  330., 2500., 2100., 3000., 2800., 3400.,
         40., 1250., 3500., 4000., 2400., 2600.,  120., 1450.,  469.,
         70., 3200.,   60.,  560.,  240.,  360., 6000., 1050., 2300.,
       4100., 5000., 3700., 1650., 2700., 4500.,  140.])

In [0]:
# before tranformation
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', None, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [0]:
def split(x):
    return x.split('/')[0]

In [0]:
df['rate']=df['rate'].astype(str).apply(split)

In [0]:
df['rate'].replace('NEW',0,inplace=True)
df['rate'].replace('-',0,inplace=True)

In [0]:
df['rate']=df['rate'].replace('None', np.nan)

In [0]:
df['rate']=df['rate'].astype(str).astype(float)

In [0]:
df['rate'].isnull().sum()

7681

In [0]:
def assign(x):
    if x>0:
        return 1
    else:
        return 0
df['rated']=df['rate'].apply(assign)

In [0]:
df['rated'].unique()

array([1, 0])

In [0]:
# separating new and training data
new_restaurants = df[df['rated'] == 0]
train_val_restaurants = df.query('rated == 1')

In [0]:
threshold = 3.75 #any restaurant with rating greater than 3.75 would have target value as 1
train_val_restaurants['target'] = train_val_restaurants['rate'].apply(lambda x: 1 if x >= threshold else 0)

<command-3058945578871469>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_val_restaurants['target'] = train_val_restaurants['rate'].apply(lambda x: 1 if x >= threshold else 0)


In [0]:
def count(x):
    return len(x.split(','))

In [0]:
#### as it have some NAN value that why very first I have to convert into str  &  then apply a function
train_val_restaurants['total_cuisines']=train_val_restaurants['cuisines'].astype(str).apply(count)
train_val_restaurants['multiple_types']=train_val_restaurants['rest_type'].astype(str).apply(count)

<command-3058945578871471>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_val_restaurants['total_cuisines']=train_val_restaurants['cuisines'].astype(str).apply(count)
<command-3058945578871471>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_val_restaurants['multiple_types']=train_val_restaurants['rest_type'].astype(str).apply(count)


In [0]:
# some important features we would be intrested in
imp_features=['online_order','book_table','location','rest_type','multiple_types','total_cuisines','listed_in(type)','approx_cost(for two people)','target']

In [0]:
data = train_val_restaurants[imp_features]

In [0]:
data.isnull().sum() # as Null data is less than 1-2% we can drop it 

online_order                     0
book_table                       0
location                         0
rest_type                      147
multiple_types                   0
total_cuisines                   0
listed_in(type)                  0
approx_cost(for two people)      0
target                           0
dtype: int64

In [0]:
data.dropna(how='any',inplace=True)

<command-3058945578871475>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(how='any',inplace=True)


In [0]:
# Splitting features by data type, any data which is object is a categorical data
cat_features= [col for col in data.columns if data[col].dtype == 'O']
num_features= [col for col in data.columns if data[col].dtype != 'O']

In [0]:
# unique features in categorical data
for feature in cat_features:
    print('{} has total {} unique features'.format(feature, data[feature].nunique()))

online_order has total 2 unique features
book_table has total 2 unique features
location has total 92 unique features
rest_type has total 87 unique features
listed_in(type) has total 7 unique features


In [0]:
#location and rest_type has too manu unique features, lets try to reduce them
cols=['location','rest_type']
for col in cols:
    print('Total feature in {} are {}'.format(col,data[col].nunique()))
    print(data[col].value_counts()/(len(data))*100)
    print('\n')
#we can see BTM is like 9% and Quickbites is like 33 percent, now we can define a threshold and put rest things as
#Other category of data

Total feature in location are 92
BTM                      9.398624
Koramangala 5th Block    5.565517
HSR                      4.828940
Indiranagar              4.361310
JP Nagar                 4.143245
                           ...   
Yelahanka                0.009692
West Bangalore           0.007269
Rajarajeshwari Nagar     0.004846
Nagarbhavi               0.002423
Peenya                   0.002423
Name: location, Length: 92, dtype: float64


Total feature in rest_type are 87
Quick Bites                   33.642663
Casual Dining                 23.301512
Cafe                           8.162919
Dessert Parlor                 4.482458
Delivery                       4.048750
                                ...    
Food Court, Beverage Shop      0.004846
Dessert Parlor, Food Court     0.004846
Dessert Parlor, Kiosk          0.004846
Bakery, Beverage Shop          0.002423
Quick Bites, Kiosk             0.002423
Name: rest_type, Length: 87, dtype: float64




In [0]:
percent=data['location'].value_counts()/len(data)*100
values=percent.values

In [0]:
len(values[values>0.4])

46

In [0]:
#### lets set Threshold value 0.4 
values=data['location'].value_counts()/len(data)*100
values

BTM                      9.398624
Koramangala 5th Block    5.565517
HSR                      4.828940
Indiranagar              4.361310
JP Nagar                 4.143245
                           ...   
Yelahanka                0.009692
West Bangalore           0.007269
Rajarajeshwari Nagar     0.004846
Nagarbhavi               0.002423
Peenya                   0.002423
Name: location, Length: 92, dtype: float64

In [0]:
threshold=0.4 #for location
imp=values[values>threshold]
imp

BTM                      9.398624
Koramangala 5th Block    5.565517
HSR                      4.828940
Indiranagar              4.361310
JP Nagar                 4.143245
Jayanagar                3.959101
Whitefield               3.808878
Marathahalli             3.416360
Bannerghatta Road        2.970537
Koramangala 7th Block    2.556212
Koramangala 6th Block    2.553789
Brigade Road             2.548944
Bellandur                2.415681
Sarjapur Road            2.069199
Koramangala 1st Block    2.064354
Ulsoor                   2.057085
Koramangala 4th Block    2.037701
Electronic City          2.020740
MG Road                  1.921399
Banashankari             1.805098
Kalyan Nagar             1.681527
Malleshwaram             1.553111
Residency Road           1.463462
Richmond Road            1.463462
Basavanagudi             1.441655
Frazer Town              1.371390
Church Street            1.322931
Brookefield              1.315662
New BEL Road             1.226013
Kammanahalli  

In [0]:
data['location']=np.where(data['location'].isin(imp.index),data['location'],'other')

<command-3058945578871483>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['location']=np.where(data['location'].isin(imp.index),data['location'],'other')


In [0]:
data['location'].nunique() #reduced the location features to 47

47

In [0]:
# for rest_type
values2=data['rest_type'].value_counts()/len(data)*100
values2

Quick Bites                   33.642663
Casual Dining                 23.301512
Cafe                           8.162919
Dessert Parlor                 4.482458
Delivery                       4.048750
                                ...    
Food Court, Beverage Shop      0.004846
Dessert Parlor, Food Court     0.004846
Dessert Parlor, Kiosk          0.004846
Bakery, Beverage Shop          0.002423
Quick Bites, Kiosk             0.002423
Name: rest_type, Length: 87, dtype: float64

In [0]:
#setting threshold as 0.3 to check number of features reduced to
len(values2[values2>0.3])

29

In [0]:
#making it 1.5 to reduce more
threshold=1.5
imp2=values2[values2>1.5]
imp2

Quick Bites           33.642663
Casual Dining         23.301512
Cafe                   8.162919
Dessert Parlor         4.482458
Delivery               4.048750
Takeaway, Delivery     3.098953
Casual Dining, Bar     2.645862
Bakery                 1.705757
Beverage Shop          1.555534
Bar                    1.550688
Name: rest_type, dtype: float64

In [0]:
data['rest_type']=np.where(data['rest_type'].isin(imp2.index),data['rest_type'],'other')

<command-3058945578871488>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rest_type']=np.where(data['rest_type'].isin(imp2.index),data['rest_type'],'other')


In [0]:
# lets see how much features we are left with now
for feature in cat_features:
    print('{} has total {} unique features'.format(feature, data[feature].nunique()))

online_order has total 2 unique features
book_table has total 2 unique features
location has total 47 unique features
rest_type has total 11 unique features
listed_in(type) has total 7 unique features


In [0]:
#feature encoding now
data_cat = data[cat_features]
for col in cat_features:
    col_encoded = pd.get_dummies(data_cat[col],prefix=col,drop_first=False)
    data_cat=pd.concat([data_cat,col_encoded],axis=1)
    data_cat.drop(col, axis=1, inplace=True)

In [0]:
data_cat.drop(['online_order_No','book_table_No'], axis=1, inplace =True)

In [0]:
#contacting with final data
data_final=pd.concat([data.loc[:,['multiple_types','total_cuisines','approx_cost(for two people)','target']],data_cat],axis=1)

In [0]:
data_final.shape

(41272, 71)

In [0]:
# Splitting the data
X = data_final.drop('target', axis=1)
y = data_final['target'].values

In [0]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)

In [0]:
model = RandomForestClassifier(n_estimators=100, min_samples_leaf=10, random_state=1)

In [0]:
# Fit the model to the data.
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=10, random_state=1)

In [0]:
# Make predictions.
predictions = model.predict(X_test)
confusion_matrix(predictions, y_test)

array([[3568, 1261],
       [ 670, 2756]])

In [0]:
accuracy_score(predictions,y_test)

0.7660811629315566

In [0]:
newDictForTest = {'multiple_types': 1,
 'total_cuisines': 3,
 'approx_cost(for two people)': 1500,
 'online_order_Yes': 1,
 'book_table_Yes': 1,
 'location_BTM': 1,
 'location_Banashankari': 0,
 'location_Banaswadi': 0,
 'location_Bannerghatta Road': 0,
 'location_Basavanagudi': 0,
 'location_Bellandur': 0,
 'location_Brigade Road': 0,
 'location_Brookefield': 0,
 'location_Church Street': 0,
 'location_Commercial Street': 0,
 'location_Cunningham Road': 0,
 'location_Domlur': 0,
 'location_Ejipura': 0,
 'location_Electronic City': 0,
 'location_Frazer Town': 0,
 'location_HSR': 0,
 'location_Indiranagar': 0,
 'location_JP Nagar': 0,
 'location_Jayanagar': 0,
 'location_Jeevan Bhima Nagar': 0,
 'location_Kalyan Nagar': 0,
 'location_Kammanahalli': 0,
 'location_Koramangala 1st Block': 0,
 'location_Koramangala 3rd Block': 0,
 'location_Koramangala 4th Block': 0,
 'location_Koramangala 5th Block': 0,
 'location_Koramangala 6th Block': 0,
 'location_Koramangala 7th Block': 0,
 'location_Koramangala 8th Block': 0,
 'location_Lavelle Road': 0,
 'location_MG Road': 0,
 'location_Malleshwaram': 0,
 'location_Marathahalli': 0,
 'location_New BEL Road': 0,
 'location_Old Airport Road': 0,
 'location_Rajajinagar': 0,
 'location_Residency Road': 0,
 'location_Richmond Road': 0,
 'location_Sarjapur Road': 0,
 'location_Shanti Nagar': 0,
 'location_Shivajinagar': 0,
 'location_St. Marks Road': 0,
 'location_Ulsoor': 0,
 'location_Vasanth Nagar': 0,
 'location_Whitefield': 0,
 'location_Wilson Garden': 0,
 'location_other': 0,
 'rest_type_Bakery': 0,
 'rest_type_Bar': 0,
 'rest_type_Beverage Shop': 0,
 'rest_type_Cafe': 0,
 'rest_type_Casual Dining': 0,
 'rest_type_Casual Dining, Bar': 0,
 'rest_type_Delivery': 0,
 'rest_type_Dessert Parlor': 0,
 'rest_type_Quick Bites': 1,
 'rest_type_Takeaway, Delivery': 0,
 'rest_type_other': 0,
 'listed_in(type)_Buffet': 1,
 'listed_in(type)_Cafes': 0,
 'listed_in(type)_Delivery': 0,
 'listed_in(type)_Desserts': 0,
 'listed_in(type)_Dine-out': 0,
 'listed_in(type)_Drinks & nightlife': 0,
 'listed_in(type)_Pubs and bars': 0}

In [0]:
#preparing dataframe for testing 
new = pd.DataFrame(newDictForTest,index=[0])

In [0]:
newPrediction = model.predict(new)

In [0]:
predProb = model.predict_proba(new)

In [0]:
predProb[0][1]

0.8019123833757231

In [0]:
import pickle
pickle.dump(model, open('SnewModel.pkl', 'wb'))

In [0]:
import mlflow

In [0]:
mlflow.sklearn.save_model(model,'/dbfs/FileStore/newModel')

In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

In [0]:
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('Naive Bayes',GaussianNB()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))

In [0]:
for name, model in models:
    print(name)
    model.fit(X_train, y_train)
    
    # Make predictions.
    predictions = model.predict(X_test)

    # Compute the error.
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(predictions, y_test))

    from sklearn.metrics import accuracy_score
    print(accuracy_score(predictions,y_test))
    print('\n')

LogisticRegression


/databricks/python/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[3486 1588]
 [ 752 2429]]
0.7165354330708661


Naive Bayes
[[3191 1501]
 [1047 2516]]
0.6913385826771653


Decision Tree
[[3862  702]
 [ 376 3315]]
0.8694124772864931


KNN


/databricks/python/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/databricks/python/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Se

[[3607  844]
 [ 631 3173]]
0.821320411871593


